In [2]:
import nltk
nltk.download('gutenberg')
from nltk.corpus import gutenberg
import pandas as pd

[nltk_data] Downloading package gutenberg to
[nltk_data]     C:\Users\anit4\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [3]:
data=gutenberg.raw('shakespeare-hamlet.txt')
with open('hamlet.txt','w') as file:
    file.write(data)

In [4]:
#data preprocessing
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer #for converting senteces into vector
from tensorflow.keras.preprocessing.sequence import pad_sequences #This function is used to make all sequences the same length by padding or truncating them.
from sklearn.model_selection import train_test_split

#load the dataset
with open('hamlet.txt','r') as file:
    data=file.read().lower()

#tokenize
tokenizer=Tokenizer()
tokenizer.fit_on_texts([data])
total_words=len(tokenizer.word_index)+1
print(total_words)


4818


In [5]:
#create input sequences
inputSequences=[]
text=data.split("\n")
for i in text:
    token_list=tokenizer.texts_to_sequences([i])[0]
    for j in range(1,len(token_list)):
        n_gram_sequence=token_list[:j+1]
        inputSequences.append(n_gram_sequence)

In [6]:
#pad sequences
max_sequence_len=max([len(x) for x in inputSequences])
inputSequences=np.array(pad_sequences(inputSequences,maxlen=max_sequence_len,padding='pre'))

In [7]:
inputSequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [8]:
#create predictors
x,y=inputSequences[:,:-1],inputSequences[:,-1]

In [9]:
import tensorflow as tf
y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [10]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

Train the lstm RNN

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,LSTM,Dense,Dropout
#Embedding : It converts word indexes (integers) into dense, learnable vectors of fixed size.
model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_len-1))
model.add(LSTM(150, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words,activation='softmax'))

In [12]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 13, 100)           481800    
                                                                 
 lstm (LSTM)                 (None, 13, 150)           150600    
                                                                 
 dropout (Dropout)           (None, 13, 150)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               100400    
                                                                 
 dense (Dense)               (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219418 (4.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [20]:
history=model.fit(x_train,y_train,epochs=100,verbose=1,validation_data=(x_test,y_test))

Epoch 1/100
644/644 [==============================] - 22s 34ms/step - loss: 2.5953 - accuracy: 0.4293 - val_loss: 11.3171 - val_accuracy: 0.0528
Epoch 2/100
644/644 [==============================] - 21s 32ms/step - loss: 2.5564 - accuracy: 0.4404 - val_loss: 11.4269 - val_accuracy: 0.0528
Epoch 3/100
644/644 [==============================] - 21s 32ms/step - loss: 2.5311 - accuracy: 0.4441 - val_loss: 11.4865 - val_accuracy: 0.0528
Epoch 4/100
644/644 [==============================] - 21s 32ms/step - loss: 2.4924 - accuracy: 0.4553 - val_loss: 11.5375 - val_accuracy: 0.0492
Epoch 5/100
644/644 [==============================] - 21s 33ms/step - loss: 2.4612 - accuracy: 0.4551 - val_loss: 11.5804 - val_accuracy: 0.0527
Epoch 6/100
644/644 [==============================] - 21s 33ms/step - loss: 2.4319 - accuracy: 0.4678 - val_loss: 11.6525 - val_accuracy: 0.0517
Epoch 7/100
644/644 [==============================] - 21s 33ms/step - loss: 2.4087 - accuracy: 0.4679 - val_loss: 11.7274 -

In [21]:
#function to predict the next word
def predict_next_word(model,tokenizer,text,max_sequence_length):
    token_list=tokenizer.texts_to_sequences([text])[0]
    token_list=pad_sequences([token_list],maxlen=max_sequence_length-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted=np.argmax(predicted,axis=1)
    for word,index in tokenizer.word_index.items():
        if index==predicted:
            return word
    return None

In [22]:
input_text="i am a"
predict_next_word(model,tokenizer,input_text,max_sequence_len)

'bearers'

In [23]:
#save the model
model.save('lstm_model.h5')
#save the tokenizer
import pickle
with open('tokenizer.pickle','wb') as handle:
    pickle.dump(tokenizer,handle,protocol=pickle.HIGHEST_PROTOCOL)